# Functions

## Defining Functions in Python

Defining functions in python is very simple:

In [9]:
def my_first_function(input):
    output = 2*input
    return output

To use the function, simply run it elsewhere once it has been defined:

In [10]:
my_first_function(5)

10

Note the indentation when writing functions. To end the function, leave a blank line and remove the indent.

In [12]:
def my_second_function(x):
    y = int(x/2)
    return y

my_second_function(12)

6

### _Docstrings_

We can write documentation for our functions, called a _docstring_, by adding writing a string in the line following the `def` call:

In [13]:
def my_documented_function(input1, input2):
    """
    This function converts inputs to string, concatenates the input, then reverses the order.
    """
    in1 = str(input1)
    in2 = str(input2)
    combi = in1+in2
    rev = combi[::-1]
    output = rev
    return output

my_documented_function(43214, 'cats')

'stac41234'

In [14]:
my_documented_function?

### _Default Values_

- We can give arguments default values when defining a function.
- Arguments taking default values become optional arguments; we do not have to pass a value each time we call the function.


In [15]:
def function_with_defaults(x, replace=" ", val=" "):
    "Casts input to string, replaces `replace` with `val`, prints result. Retuns None"
    y = str(x)
    y = y.replace(replace, val)
    print(y)

function_with_defaults("1. I love dogs.")
function_with_defaults("2. I love dogs.", " dogs.")
function_with_defaults("3. I love dogs.", val="_")
function_with_defaults("4. I love dogs.", "d", "b")
function_with_defaults("5. I love dogs.", "dog", "cat")

1. I love dogs.
2. I love 
3._I_love_dogs.
4. I love bogs.
5. I love cats.


### Namespaces

Recall the difference between local and global namespace.

- Variables named within functions are not accessible outside the function.
- When a variable is called within a function, the program first checks if it is defined locally, then checks if it is defined globally.

In [17]:
def function_with_local():
    some_local_variable = 12
    
print(some_local_variable)

NameError: name 'some_local_variable' is not defined

In [3]:
a = "Global A"

def function1():
    print(a)
    
def function2():
    a = "Local A"
    print(a)
    
def function3(a):
    print(a)

function1()
function2()
function3("Argument A")

Global A
Local A
Argument A


### Namespaces Take Away

When defining variables within the global environment, use _unique, specific and informative names_. When working within functions, give generic names that inform what the argument or variable is doing.

# Applying Functions to Vectors

We go over a variety of ways in which you may apply a function to a `pandas.Series` or `pandas.DataFrame`.

- Transformations:
    - Element-wise Operations
    - Cumulative Operations
- Summaries:
    - Point Summaries
    - Grouped Summaries

## Element-wise Operations on a Series

We can use the `pd.Series.apply()` method to apply a function element-wise to a pandas Series.

In [6]:
import pandas as pd

ser = pd.Series(range(0, 12, 2)) # range(start, stop, step)
ser

0     0
1     2
2     4
3     6
4     8
5    10
dtype: int64

In [18]:
def square(x):
    y = x**2
    return y

ser.apply(square)

0      0
1      4
2     16
3     36
4     64
5    100
dtype: int64

In [19]:
def exponentiate(x, e):
    y = x**e
    return y

ser.apply(lambda x: exponentiate(x, 3))

0       0
1       8
2      64
3     216
4     512
5    1000
dtype: int64

In [ ]:
ser.apply(lambda x: x**3)

In [ ]:
e = 1/2
ser.apply(lambda x: x**e)

## Cumulative Operations on a Series

In order to use cumulative operations, we can either use a `cum` function, or the `pd.Series.expanding` method.

In [ ]:
ser.cumsum()

In [ ]:
ser.expanding()

In [ ]:
ser.expanding().sum()

In [ ]:
ser.expanding(2).sum() # We can set the minimum period within the expand function.

We can also use apply with a DataFrame. In this case, each row (axis=0) or column (axis=1) is treated as an element.

In [ ]:
df = pd.DataFrame({
    'col1': pd.np.random.randint(-100, 100, 5),
    'col2': pd.np.random.randint(-100, 100, 5),
    'col3': pd.np.random.randint(-100, 100, 5)
})
df.head()

In [ ]:
df.apply(lambda x: x.mean(), axis=0)

In [ ]:
df.apply(lambda x: x.sum(), axis=1)

In [ ]:
df.applymap(lambda x: abs(x)**0.5)

## Point Summaries

We have already looked at a number of point summary functions in the previous week.

- `pd.Series.mean()`
- `pd.Series.sum()`

We do not spend more time on them here.

## Grouped Summaries

The syntax for group summaries is [explained in detail in the lecture](https://muhark.github.io/dpir-intro-python/Week3/lecture.html#/groupby-syntax-simple-group-operations).

In [ ]:
df = pd.read_feather("../Week2/data/bes_data_subset_week2.feather")

In [ ]:
df.groupby('region')['Age'].mean()

In [ ]:
df.groupby('region')[['Age']].mean() # List indexer returns a DataFrame of width 1.

We can pass custom functions to the groupby object by using `apply`

In [ ]:
df.groupby(['region', 'Constit_Code'])['Age'].apply(lambda x: f"{int(x.min())}-{int(x.max())}").rename('Age Range')

We can pass multiple functions by using the `agg` function.

In [ ]:
df.groupby(['region', 'Constit_Code'])[['Age']].agg([pd.np.mean, len])

We can also apply a single function to multiple columns simultaneously:

In [ ]:
df.groupby(['region', 'Constit_Code'])[['k03', 'y06', 'y09', 'y11']].apply(lambda x: x.mode().iloc[0])

And finally, we can map different functions to different columns using the `agg()` function with a dictionary:

In [ ]:
def group_mode(x):
    "Function for extracting first modal value from pandas groupby object"
    m = x.value_counts().index[0]
    return m

def gender_proportion(x):
    m = x.apply(lambda e: 1 if e=="Female" else 0)
    m = m.astype(int).mean()
    return m


df.groupby(['region', 'Constit_Code']).agg({'k03': group_mode,
                                            'y06': group_mode,
                                            'y09': gender_proportion,
                                            'Age': ['min', 'max']})

Note: to index the above DataFrame, you will need some fancy indexers, namely `pd.IndexSlice`.

For more general notes, see: https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-indexing-with-hierarchical-index

In [ ]:
temp = df.groupby(['region', 'Constit_Code']).agg({'k03': group_mode,
                                                   'y06': group_mode,
                                                   'y09': gender_proportion,
                                                   'Age': ['min', 'max']})

In [ ]:
idx = pd.IndexSlice
temp.loc[idx['East Midlands', :], idx[:, 'group_mode']]

We can also use apply with a DataFrame. In this case, each row (axis=0) or column (axis=1) is treated as an element.

# Combining Datasets

We look at two commands in particular. For an in-depth explanation, see: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [ ]:
df1 = df.loc[:, ['finalserialno', 'Age', 'y09']]
df2 = df.loc[:, ['finalserialno', 'y06']]

## Using `pd.concat`

In [ ]:
print(df1.shape)
print(df2.shape)
print(pd.concat([df1, df2], axis=0, sort=False).shape)
pd.concat([df1, df2], axis=0, sort=False)

In [ ]:
print(df1.shape)
print(df2.shape)
print(pd.concat([df1, df2], axis=1).shape)

pd.concat([df1, df2], axis=1)

In [ ]:
df3 = df.loc[:2130, ['finalserialno', 'a02']]

In [ ]:
pd.concat([df1, df2, df3], axis=1)

## `pd.merge`

In [ ]:
pd.merge(df1, df2, on="finalserialno")

In [ ]:
df4 = df.loc[30:, ['finalserialno', 'y11']]

In [ ]:
print(df3.index)
print(df4.index)
for join in ['inner', 'left', 'right', 'outer']:
    print(pd.merge(df3, df4, how=join, on="finalserialno").shape)

In [ ]:
df4 = df4.rename({'finalserialno':'serialno'}, axis=1).set_index('serialno')
df4

In [ ]:
pd.merge(df3, df4, how="outer", left_on="finalserialno", right_index=True)

# Melting and Pivoting

In [ ]:
long_df = pd.DataFrame({
    "Constituency": ['Oxford West', 'Oxford East']*4,
    "Year": [2010, 2010, 2015, 2015, 2017, 2017, 2019, 2019],
    "Party": ["Labour", "Tory"]*2+["Labour", "LibDem"]*2
})

long_df

In [ ]:
wide_df = long_df.pivot(index="Constituency", columns="Year", values="Party")
wide_df

In [ ]:
wide_df.reset_index().melt(id_vars="Constituency", value_vars=[2010, 2015, 2017, 2019], var_name="Year")